In [1]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from statsmodels.tsa.seasonal import seasonal_decompose
from tqdm import tqdm
import os
import pickle

In [2]:
class Predictor:
    def __init__(self, point_hash: str):
        self.point_hash = point_hash
        self.result_per_day = None
        self.result_per_hour = None
        self.resid_regr = None
        self.train_mae = None
        self.val_mae = None
        self.val_prediction = None
    
    def fit(self, series: pd.Series):
        df = series.to_frame().reset_index()
        
        # Average number of publications during a day.
        df["timestamp"] = df["timestamp"].apply(datetime.datetime.fromtimestamp)
        df["year_day"] = df["timestamp"].apply(lambda x: x.day_of_year)
        series_per_day = df.groupby("year_day").apply(lambda x: x["n_pubs"].mean())
        
        # Decompose dayly trend and seasonal components.
        result_per_day = seasonal_decompose(series_per_day, period=31, model='additive', extrapolate_trend="freq")
        self.result_per_day = result_per_day
        
        # Substract dayly trend and seasonal components from original series 
        to_substract = (result_per_day.trend + result_per_day.seasonal).to_frame("t+s").reset_index()
        df2 = df.merge(to_substract, on="year_day")
        df2 = df2.set_index("timestamp")
        df2["n_pubs"] -= df2["t+s"]
        series_per_hour = df2["n_pubs"]
        
        # Decompose hourly trend and seasonal components.
        result_per_hour = seasonal_decompose(series_per_hour, period=24, model='additive', extrapolate_trend="freq")
        self.result_per_hour = result_per_hour
        
        # Compose dayly and hourly components.
        df3 = (result_per_hour.trend + result_per_hour.seasonal).to_frame("amount").reset_index()
        df3["year_day"] = df3["timestamp"].apply(lambda x: x.day_of_year)
        to_add = (result_per_day.trend + result_per_day.seasonal).to_frame("t+s").reset_index()
        df3 = df3.merge(to_add, on="year_day")
        df3["amount"] += df3["t+s"]
        df3 = df3.set_index("timestamp")
        result = df3["amount"]
        self.result = result
        
        # Calculate residual.
        series_ = series.to_frame().reset_index()
        series_["timestamp"] = series_["timestamp"].apply(datetime.datetime.fromtimestamp)
        series_ = series_.set_index("timestamp")
        series_ = series_["n_pubs"]
        residual = series_ - result
        
        # Train model to predict residual.
        df_ = residual.to_frame("amount").reset_index()
        df_["hour"] = df_["timestamp"].apply(lambda x: x.hour)
        df_["year_day"] = df_["timestamp"].apply(lambda x: x.day_of_year)
        x = df_[["hour", "year_day"]].to_numpy()
        y = df_["amount"].to_numpy()
        resid_regr = RandomForestRegressor()
        resid_regr.fit(x, y)
        self.resid_regr = resid_regr
        
        # Evaluate training.
        resid_pred = resid_regr.predict(x)
        result_ = result.copy()
        result_ += resid_pred
        result_[result_ < 0] = 0
        result_ = result_.apply(np.round)
        self.train_mae = mean_absolute_error(result_, series_)

    
    def evaluate(self, val_series: pd.Series):
        val_series_ = val_series.to_frame().reset_index()
        val_series_["timestamp"] = val_series_["timestamp"].apply(datetime.datetime.fromtimestamp)
        val_series_["timestamp"] -= datetime.timedelta(days=365) # Hard-coded fix to enable use of previous year data.
        val_series_ = val_series_.set_index("timestamp")
        val_series_ = val_series_["n_pubs"]
        
        start = val_series_.index.min()
        end = val_series_.index.max()
        
        # Select composed comonents data for required time range.
        val_result_ = self.result.copy()
        val_result_ = val_result_.loc[start: end]
        
        # Predict residuals.
        df_ = val_series_.to_frame("amount").reset_index()
        df_["hour"] = df_["timestamp"].apply(lambda x: x.hour)
        df_["year_day"] = df_["timestamp"].apply(lambda x: x.day_of_year)
        val_x = df_[["hour", "year_day"]].to_numpy()
        val_resid_pred = self.resid_regr.predict(val_x)
        val_resid_pred = pd.Series(index=df_["timestamp"], data=val_resid_pred)
        
        # Compose result
        val_result_ += val_resid_pred
        val_result_[val_result_ < 0] = 0
        val_result_ = val_result_.apply(np.round)
        self.val_mae = mean_absolute_error(val_result_, val_series_[val_result_.index])
        self.val_prediction = val_result_
        
    def save(self, dir):
        if not os.path.exists(dir):
            os.makedirs(dir, exist_ok=True)
        
        with open(os.path.join(dir, self.point_hash + ".pickle"), "wb") as f:
            pickle.dump(self, f)        

In [3]:
with open("data/selected_data.pickle", "rb") as f:
    data = pickle.load(f)

In [4]:
train = data["train"]
train_points = train["points"]
train_groups = train["groups"]

val = data["val"]
val_points = val["points"]
val_groups = val["groups"]

In [5]:
train_maes = []
val_maes = []

for SAMPLE_ID in tqdm(range(len(train_groups))):
    train_point = train_points[SAMPLE_ID]
    val_point = val_points[SAMPLE_ID]
    assert train_point == val_point
    train_series = train_groups[SAMPLE_ID]
    val_series = val_groups[SAMPLE_ID]
    model = Predictor(train_point)
    model.fit(train_series)
    model.evaluate(val_series)
    train_maes.append(model.train_mae)
    val_maes.append(model.val_mae)
    model.save("./models")

  0%|          | 0/1666 [00:00<?, ?it/s]

100%|██████████| 1666/1666 [25:33<00:00,  1.09it/s]


In [7]:
pd.Series(val_maes).describe()

count    1666.000000
mean        0.543562
std         1.621069
min         0.000000
25%         0.108871
50%         0.222446
75%         0.499664
max        41.350806
dtype: float64